# Building and Deploying Deep Learning Applications with TensorFlow

In [10]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
tf1.disable_v2_behavior()

In [7]:
# Load Training data set from CSV file
training_data_df = pd.read_csv("./ex_files/03/sales_data_training.csv", dtype=float)

# Pull out columns for X (data to train with) and Y (value to predict)
x_training = training_data_df.drop('total_earnings', axis=1).values
y_training = training_data_df[['total_earnings']].values

# Load testing data set from CSV file
test_data_df = pd.read_csv("./ex_files/03/sales_data_test.csv")

# Pull out columns for X (data to train with) and Y (value to predict)
x_testing = test_data_df.drop('total_earnings', axis=1).values
y_testing = test_data_df[['total_earnings']].values


# All data needs to be scaled to a small range like 0 to 1 for the neural
# network to work well. Create scalers for the inputs and outputs
x_scaler = MinMaxScaler(feature_range=(0, 1)) 
y_scaler = MinMaxScaler(feature_range=(0, 1))

# Scale both the training input and outputs
x_scaled_training = x_scaler.fit_transform(x_training)
y_scaled_training = y_scaler.fit_transform(y_training)

# It's important that the training and test data are scaled with the same scaler
x_scaled_testing = x_scaler.transform(x_testing) 
y_scaled_testing = y_scaler.transform(y_testing)

print(y_scaled_testing.shape)
print(x_scaled_testing.shape)

print("Note: Y values were scaled by multiplying by {:.10f} and adding {:.4f}".format(y_scaler.scale_[0],
     y_scaler.min_[0]))

(400, 1)
(400, 9)
Note: Y values were scaled by multiplying by 0.0000036968 and adding -0.1159


## Define the model structure

In [11]:
# Define model parameters
learning_rate = 0.001
training_epochs = 100
display_step = 5

# Define how many inputs and outputs are in our neural networks
number_of_inputs = 9
number_of_outputs = 1

# Define how many neurons we want in each layer of our neural network
layer_1_nodes = 50
layer_2_nodes = 100
layer_3_nodes = 50

# Selection One: Define the layers of the neural network itself


tf1.reset_default_graph()
# Input layer
with tf1.variable_scope('input'):
    x = tf1.placeholder(tf.float32, shape=(None, number_of_inputs))
    
# Layer 1
with tf1.variable_scope('layer_1'):
    weights = tf1.get_variable(name='weigths1', shape=[number_of_inputs, layer_1_nodes],
                             initializer=tf.initializers.GlorotUniform())
    biases = tf1.get_variable(name='biases1', shape=[layer_1_nodes],
                            initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(x, weights) + biases)
    
    
# Layer 2
with tf1.variable_scope('layer_2'):
    weights = tf1.get_variable(name='weights2', shape=[layer_1_nodes, layer_2_nodes])
    biases = tf1.get_variable(name='biases2', shape=[layer_2_nodes],
                             initializer=tf.initializers.GlorotUniform())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)
    
    
# Layer 3
with tf1.variable_scope('layer_3'):
    weights = tf1.get_variable(name='weights3', shape=[layer_2_nodes, layer_3_nodes])
    biases = tf1.get_variable(name='biases3', shape=[layer_3_nodes],
                             initializer=tf.initializers.GlorotUniform())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)
    
    
# Output layer
with tf1.variable_scope('output'):
    weights = tf1.get_variable(name='weights4', shape=[layer_3_nodes, number_of_outputs])
    biases = tf1.get_variable(name='biases4', shape=[number_of_outputs],
                            initializer=tf.initializers.GlorotUniform())
    prediction = tf.nn.relu(tf.matmul(layer_3_output, weights) + biases)
    
# Section two: Define the cost function of the neural network that will measure the predict
with tf1.variable_scope('cost'):
    y = tf1.placeholder(tf.float32, shape=(None,1))
    cost = tf.reduce_mean(tf.compat.v1.squared_difference(prediction, y))
    
# Section three: define the optimize function that will be run to optimize the neural network

with tf1.variable_scope('train'):
    optimizer = tf1.train.AdamOptimizer(learning_rate).minimize(cost)
    
    
# Create a summary operation to log the progress of the network
with tf1.variable_scope('logging'):
    tf1.summary.scalar('current_cost', cost)
    summary = tf1.summary.merge_all()
    
    
    
    
saver = tf1.train.Saver()
    
    
# Initialize a session so that we can run TensorFlow operations
with tf1.Session() as session:
    
    # Run the global variable initializer to initialize all variables and layers
    # of the graph
    session.run(tf1.global_variables_initializer())
    
    # Instead load them from disk
#     saver.restore(session, "logs/trained_model.ckpt")
    
    
    # Create log file writers to record training progress.
    # We'll store training and testing log data separately.
    training_writer = tf1.summary.FileWriter('./logs/training', session.graph)
    testing_writer = tf1.summary.FileWriter('./logs/testing', session.graph)
    
    
    # Run the optimizer over and over to train the network
    # One epoch is one full run through the training data set
    for epoch in range(training_epochs):
        
        # Feed in the training data and do one step of neural network training
        session.run(optimizer, feed_dict={x: x_scaled_training, y: y_scaled_training})
        
        # Every 5 training steps, log our progress
        if epoch % 5 == 0:
            training_cost, training_summary = session.run([cost, summary], 
                                        feed_dict={x: x_scaled_training, 
                                                  y: y_scaled_training})
            testing_cost, testing_summary = session.run([cost, summary],
                                                        feed_dict={x: x_scaled_testing,
                                                       y: y_scaled_testing})
            
            # Write the current training status to the log files
            training_writer.add_summary(training_summary, epoch)
            testing_writer.add_summary(testing_summary, epoch)
            
            
            print(epoch, training_cost, testing_cost)
        
    # Training is now complete!
    print("Training is complete!")


    final_training_cost = session.run(cost, feed_dict={x: x_scaled_training, 
                                          y: y_scaled_training})
    final_testing_cost = session.run(cost, feed_dict={x: x_scaled_testing,
                                       y: y_scaled_testing})

    print("Final Training cost: {}".format(final_training_cost))
    print("Final Testing cost: {}".format(final_testing_cost))
    
    
    
    save_path = saver.save(session, "logs/trained_model.ckpt")
    print("Model saved: {}".format(save_path))

2022-04-07 22:52:13.167824: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-07 22:52:13.168490: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-07 22:52:13.168702: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lucas-Inspiron-5566): /proc/driver/nvidia/version does not exist
2022-04-07 22:52:13.171002: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


0 0.1136593 0.12273928
5 0.1136593 0.12273928
10 0.1136593 0.12273928
15 0.1136593 0.12273928
20 0.1136593 0.12273928
25 0.1136593 0.12273928
30 0.1136593 0.12273928
35 0.1136593 0.12273928
40 0.1136593 0.12273928
45 0.1136593 0.12273928
50 0.1136593 0.12273928
55 0.1136593 0.12273928
60 0.1136593 0.12273928
65 0.1136593 0.12273928
70 0.1136593 0.12273928
75 0.1136593 0.12273928
80 0.1136593 0.12273928
85 0.1136593 0.12273928
90 0.1136593 0.12273928
95 0.1136593 0.12273928
Training is complete!
Final Training cost: 0.11365929991006851
Final Testing cost: 0.122739277780056
Model saved: logs/trained_model.ckpt
